# 2 Introduction to Dask

In [ ]:
import warnings

warnings.filterwarnings("ignore")
import dask.dataframe as dd

import dask
import matplotlib.pyplot as plt
import pandas as pd
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster

ddf = dd.read_csv(
    "data/parking_violations/Parking_Violations_Issued_-_Fiscal_Year_2017.csv",
    dtype={"House Number": "object", "Time First Observed": "object"},
)


In [ ]:
missing_values = ddf.isnull().sum()
missing_count_dag = (missing_values / ddf.index.size) * 100

with ProgressBar():
    missing_count = missing_count_dag.compute()
    columns_to_drop = missing_count[missing_count > 60].index
    ddf_dropped_dag = ddf.drop(columns_to_drop, axis=1)
    ddf_dropped_persisted = ddf.drop(columns_to_drop, axis=1).persist()


In [ ]:
missing_values = ddf.isnull().sum()
missing_count_dag = (missing_values / ddf.index.size) * 100

with (
    LocalCluster(n_workers=6, processes=True, memory_limit="2GB") as cluster,
    Client(cluster) as client,
):
    print(f"Dashboard link:", {client.dashboard_link})
    missing_count = missing_count_dag.compute()
    columns_to_drop = missing_count[missing_count > 60].index
    ddf_dropped_dag = ddf.drop(columns_to_drop, axis=1)
    ddf_dropped_persisted = ddf.drop(columns_to_drop, axis=1).persist()
